### Runtime & environment

In [1]:
!pip -q install py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 21.1 MB/s eta 0:00:00


### Imports, config, seeds, class names

In [2]:
import os, math, glob, numpy as np, tensorflow as tf, pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

# Reproducibility & performance
SEED = 1337
tf.keras.utils.set_random_seed(SEED)
AUTO = tf.data.AUTOTUNE
tf.config.optimizer.set_jit(False) 

# Hyperparameters
BATCH_TRAIN = 64
BATCH_TEST  = 64
EPOCHS      = 100
LR          = 1e-2
MOMENTUM    = 0.9
WEIGHT_DECAY = 1e-6  # L2 via kernel_regularizer
NUM_CLASSES = 10
CLASS_NAMES = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']


2025-09-19 18:08:23.367624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758305303.709203      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758305303.801447      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Loads official CIFAR-10 (train/test) via Keras

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test  = keras.utils.to_categorical(y_test,  NUM_CLASSES)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
(50000, 32, 32, 3) (50000, 10) (10000, 32, 32, 3) (10000, 10)


### Augmentations & tf.data pipelines

In [4]:
train_aug = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomTranslation(0.05, 0.05, fill_mode="reflect"),
    layers.RandomZoom(0.05, 0.05, fill_mode="reflect"),
    layers.RandomContrast(0.1),
    layers.RandomBrightness(0.1),
])

def preprocess(x, y=None):
    x = tf.cast(x, tf.float32) / 255.0
    return (x, y) if y is not None else x

def make_ds(x, y, train=False, batch=100):
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    if train:
        ds = ds.shuffle(10000, seed=SEED, reshuffle_each_iteration=True)
        ds = ds.map(lambda a,b: (train_aug(a), b), num_parallel_calls=AUTO)
    ds = ds.map(lambda a,b: (preprocess(a), b), num_parallel_calls=AUTO)
    ds = ds.batch(batch).prefetch(AUTO)
    return ds

train_ds = make_ds(x_train, y_train, train=True,  batch=BATCH_TRAIN)
test_ds  = make_ds(x_test,  y_test,  train=False, batch=BATCH_TEST)  # used as validation proxy for Kaggle


I0000 00:00:1758305329.706589      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758305329.707314      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


### Cutout

In [5]:
USE_CUTOUT = False  # set True for a small boost

def cutout(img, length=8):
    H = tf.shape(img)[0]; W = tf.shape(img)[1]
    y = tf.random.uniform([], 0, H, dtype=tf.int32)
    x = tf.random.uniform([], 0, W, dtype=tf.int32)
    y1 = tf.clip_by_value(y - length//2, 0, H); y2 = tf.clip_by_value(y + length//2, 0, H)
    x1 = tf.clip_by_value(x - length//2, 0, W); x2 = tf.clip_by_value(x + length//2, 0, W)
    mask = tf.ones([y2-y1, x2-x1, 3], dtype=img.dtype)
    paddings = [[y1, H - y2], [x1, W - x2], [0, 0]]
    mask = tf.pad(mask, paddings, constant_values=0)
    return img * (1 - mask)

if USE_CUTOUT:
    def _train_map(a, b):
        a = cutout(a, length=8)
        return a, b
    train_ds = train_ds.map(_train_map, num_parallel_calls=AUTO).prefetch(AUTO)


### Advanced CNN

In [6]:
L2 = keras.regularizers.l2(WEIGHT_DECAY)

def conv_bn_relu(x, filters, k=3, bn=False):
    x = layers.Conv2D(filters, k, padding="same", use_bias=not bn,
                      kernel_regularizer=L2)(x)
    if bn:
        x = layers.BatchNormalization()(x)
    return layers.ReLU()(x)

inputs = keras.Input(shape=(32, 32, 3))
x = inputs

# Block 1: 3 convs -> MaxPool
x = conv_bn_relu(x, 64*2, bn=True)   # conv1 + BN
x = conv_bn_relu(x, 64*2)            # conv2
x = conv_bn_relu(x, 64*2)            # conv3
x = layers.MaxPooling2D(2)(x)

# Block 2: 3 convs -> MaxPool -> Dropout
x = conv_bn_relu(x, 128*2, bn=True)  # conv4 + BN
x = conv_bn_relu(x, 128*2)           # conv5
x = conv_bn_relu(x, 128*2)           # conv6
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.2)(x)

# Block 3: 3 convs -> MaxPool -> Dropout
x = conv_bn_relu(x, 256*2, bn=True)  # conv7 + BN
x = conv_bn_relu(x, 256*2)           # conv8
x = conv_bn_relu(x, 256*2)           # conv9
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.2)(x)

# Classifier: 4*4*512 = 8192 -> 8192 -> 4096 -> 10
x = layers.Flatten()(x)
x = layers.Dense(4096*2, activation="relu", kernel_regularizer=L2)(x)
x = layers.Dense(2048*2, activation="relu", kernel_regularizer=L2)(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 128)    │         3,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 256)    │       294,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_4 (ReLU)                  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_5 (ReLU)                  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 8, 8, 512)      │     1,179,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_6 (ReLU)                  │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_7 (ReLU)                  │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 108,393,098 (413.49 MB)

 Trainable params: 108,391,306 (413.48 MB)

 Non-trainable params: 1,792 (7.00 KB)

### Optimizer & compile (SGD + momentum)

In [7]:
opt = keras.optimizers.SGD(learning_rate=LR, momentum=MOMENTUM, nesterov=False)

model.compile(
    optimizer=opt,
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.0),
    metrics=["accuracy"]
)


### Callbacks

In [8]:
ckpt = keras.callbacks.ModelCheckpoint(
    filepath="adv_best.weights.h5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
)
plateau = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.1, patience=10, min_lr=1e-5, verbose=1
)
es = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=20, restore_best_weights=True
)
log = keras.callbacks.CSVLogger("adv_training_log.csv")


### Train

In [9]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS,
    callbacks=[ckpt, plateau, es, log]
)
print("Final val_acc (official test):", round(float(history.history["val_accuracy"][-1]), 4))


Epoch 1/100


I0000 00:00:1758305338.021077      73 service.cc:148] XLA service 0x7a580c005270 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758305338.022913      73 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1758305338.022935      73 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1758305338.646138      73 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/782 ━━━━━━━━━━━━━━━━━━━━ 3:34:58 17s/step - accuracy: 0.1250 - loss: 2.4823

I0000 00:00:1758305350.046168      73 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


782/782 ━━━━━━━━━━━━━━━━━━━━ 132s 148ms/step - accuracy: 0.3639 - loss: 1.7466 - val_accuracy: 0.5449 - val_loss: 1.3814 - learning_rate: 0.0100
Epoch 2/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 102s 131ms/step - accuracy: 0.6222 - loss: 1.0737 - val_accuracy: 0.4842 - val_loss: 1.8431 - learning_rate: 0.0100
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 107s 136ms/step - accuracy: 0.7187 - loss: 0.8145 - val_accuracy: 0.7335 - val_loss: 0.8005 - learning_rate: 0.0100
Epoch 4/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 106s 135ms/step - accuracy: 0.7727 - loss: 0.6687 - val_accuracy: 0.7609 - val_loss: 0.7205 - learning_rate: 0.0100
Epoch 5/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 106s 135ms/step - accuracy: 0.7969 - loss: 0.5965 - val_accuracy: 0.7695 - val_loss: 0.7171 - learning_rate: 0.0100
Epoch 6/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 106s 136ms/step - accuracy: 0.8235 - loss: 0.5185 - val_accuracy: 0.8045 - val_loss: 0.5973 - learning_rate: 0.0100
Epoch 7/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 102s 131ms/step - accuracy: 0

### Evaluate on official test (your Kaggle score estimate)

In [10]:
loss, acc = model.evaluate(test_ds, verbose=0)
print("Official CIFAR-10 test accuracy (Kaggle-score estimate):", round(float(acc), 4))


Official CIFAR-10 test accuracy (Kaggle-score estimate): 0.9046


### locate & extract test.7z

In [11]:
import py7zr, os, glob

input_dir = "/kaggle/input/cifar-10"
test_7z = os.path.join(input_dir, "test.7z")

if os.path.exists(test_7z):
    with py7zr.SevenZipFile(test_7z, mode='r') as z:
        z.extractall(path=".")
    print("Extracted test images:", len(glob.glob("test/*.png")))
else:
    print("test.7z not found at", test_7z)


Extracted test images: 300000


### Build submission.csv

In [12]:
from PIL import Image

def load_png(fp):
    im = Image.open(fp).convert("RGB")
    # Should already be 32x32; resize defensively just in case
    im = im.resize((32,32), resample=Image.BILINEAR)
    arr = np.asarray(im).astype(np.float32) / 255.0
    return arr

test_files = sorted(glob.glob("test/*.png"), key=lambda p: int(os.path.splitext(os.path.basename(p))[0]))

BATCH = 1024
pred_labels = []
for i in range(0, len(test_files), BATCH):
    batch_files = test_files[i:i+BATCH]
    batch = np.stack([load_png(f) for f in batch_files], axis=0)
    probs = model.predict(batch, verbose=0)
    idx  = np.argmax(probs, axis=1)
    pred_labels.extend([CLASS_NAMES[j] for j in idx])

ids = [int(os.path.splitext(os.path.basename(f))[0]) for f in test_files]
sub = pd.DataFrame({"id": ids, "label": pred_labels})
sub.to_csv("submission.csv", index=False)
sub.head()


,id,label
0,1,automobile
1,2,airplane
2,3,automobile
3,4,ship
4,5,bird


### quick sanity checks on submission

In [13]:
# Check format and label set
assert set(sub.columns)=={"id","label"}
assert set(sub["label"].unique()).issubset(set(CLASSES))
assert sub["id"].min()==1 and sub["id"].max()==300000 and len(sub)==300000
print("Submission looks well-formed:", sub.shape, "rows")


NameError: name 'CLASSES' is not defined